# Shared Task: Critical thinking generation
## Loadind Data


In [ ]:
pip install -q social_agents/

In [ ]:
pip show social_agents

In [ ]:
import tqdm
# Reading the data
import json
import pandas as pd
import social_agents

from social_agents.utils import get_st_data


for key, line in tqdm.tqdm(get_st_data("sample").items()):
    print(key)

    print(line['intervention'])
    input_text = line['intervention']
    break


In [ ]:
%%capture --no-stderr
%pip install --quiet -U langchain_openai langchain_core langchain_community tavily-python

## Prerequisites
env, LLM, ...

In [ ]:
import os, getpass
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")

_set_env("OPENAI_API_KEY")
_set_env("LANGCHAIN_API_KEY")
_set_env("MISTRAL_API_KEY")

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "shared_task_critical_questions"


##  1. Zero Shot LLM to start with o3-mini

In [ ]:

from social_agents.graph_tools import BasicCQModel
model_name = "o3-mini"

basic_agent = BasicCQModel(llm_name = model_name, temperature=None ) #interrupt_before=['human_feedback'], checkpointer=memory)


display(Image(basic_agent.graph.get_graph(xray=1).draw_mermaid_png()))

# RUN
# basic_agent.run_experiment()



### Evaulate 

- Run shell 

```shell

python3 eval_scripts/evaluation.py \
    --metric similarity \
    --input_path data_splits/validation.json \
    --submission_path output/output_o3-mini_temperatureNA.json \
    --threshold 0.6 

```

- OUTPUT : output/output_o3-mini_temperatureNA._eval_similarity_06.json



**Overall count**

| **Questions Labels** |  **#**  | **ratio** |
|:--------------------|-------:|---------:|
| useful               | **329** |  **0,59** |
| unhelpful            |      63 |      0,11 |
| Invalid              |       8 |      0,01 |
| Not able to evaluate |     158 |      0,28 |
| **Total**            | **558** |         1 |


**Overall count within each argument**

| **n/3 useful questions per arg** | **# of arguments** | **ratio** |
|:-------------------------------:|-------------------:|:---------:|
|                               0/3 |                 17 |      0,10 |
|                               1/3 |                 51 |      0,27 |
|                               2/3 |             **76** |      0,40 |
|                               3/3 |                 42 |      0,23 |
| **Total**                       |            **186** |         1 |

## Social Agents

In [ ]:
import pandas as pd

EXPERIMENT_SETTINGS_FILE_PATH = "output/elbaff_experiment/experiment_settings.csv"
experiment_settings = pd.read_csv(EXPERIMENT_SETTINGS_FILE_PATH)


llm_name = "gpt-4o-mini-2024-07-18"# "o3-mini-2025-01-31"
temperature = 0.7

exp_row = experiment_settings.iloc[9].to_dict()
exp_row

In [ ]:
import ast
exp_row["experiment_name"].format(llm_name= llm_name)

list(ast.literal_eval(exp_row["traits"]))

In [ ]:

import ast
from social_agents import objects
from social_agents.graph_tools import  CQSTAbstractAgent, SocialAgentBuilder
import json 

#for _, exp_row in experiment_settings.items():

social_agent = SocialAgentBuilder(
    model_thread_id=exp_row["thread_id"],
    llm_name = llm_name,
    llm_num = exp_row["number_of_agents"],
    experiment_name= exp_row["experiment_name"].format(llm_name= llm_name),
    temperature=temperature,
    collaborative_strategy=list(ast.literal_eval(exp_row["strategies"])),
    agent_trait_lst=list(ast.literal_eval(exp_row["traits"])))

id_= "123"
params = {"input_arg": "To be or not to be this is the question. Is it?"}

questions = []
id_ = "123"
config = {"configurable": {"thread_id": f"{social_agent.model_thread_id}_{id_}"}}

params = {
    "input_arg": "To be or not to be this is the question. Is it?",
}

for event in social_agent.graph.stream(params, config=config, stream_mode="values"):
    # Review
    _labels = event.get("final_cq", "")
    if _labels:
        questions.append(_labels)
        # Save the final state
        with open(f"{CQSTAbstractAgent.ROOT_FOLDER}{social_agent.experiment_name}_arg{id_}.json", "w") as f:
          json.dump(objects.state_to_serializable(event), f,indent=2)
        break
assert len(questions) == 1


In [ ]:
from social_agents import objects




In [ ]:
from collections import Counter

my_list = ["debate", "debate", "reflect"]
counts = Counter(my_list)
print(counts)        # Output: Counter({2: 3, 1: 1, 3: 1, 4: 1, 5: 1})
print(counts["debate"])     # Output: 3


In [ ]:
{"configurable": {"thread_id": f"{social_agent.model_thread_id}_{id_}"}}.items()

In [ ]:
graph_obj = social_agent.graph.get_graph(xray=1)
mermaid_code = graph_obj.draw_mermaid()  # Get the textual diagram
print(mermaid_code)
display(Image(social_agent.graph.get_graph().draw_mermaid_png()))


In [ ]:
from IPython.display import Image, display
display(Image(social_agent.graph.get_graph(xray=1).draw_mermaid_png()))



In [ ]:
_invoke_graph(params, id_)

In [ ]:
response = app.invoke({"collaborative_strategy": collaborative_strategy, "input_arg": "I am human therefore I am mortal"}, config)
